<a href="https://colab.research.google.com/github/R1M1N/ZTM_tf2/blob/main/06_Transfer_Learning_Scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Start with a small data 1% or 10% and build a small model.
- Next build a big model with small data
- Increase the data
- Build the complete model

# What we're going to cover
####We're going to go through the follow with TensorFlow:

- Downloading and preparing 10% of the Food101 data (10% of training data)
- Training a feature extraction transfer learning model on 10% of the Food101 training data
- Fine-tuning our feature extraction model
- Saving and loaded our trained model
- Evaluating the performance of our Food Vision model trained on 10% of the training data
-     Finding our model's most wrong predictions
- Making predictions with our Food Vision model on custom images of food

In [1]:
# Check for GPU
!nvidia-smi

Wed Nov 15 03:35:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Get the helper functions file
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Perform important imports
import tensorflow as tf
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir, compare_historys


--2023-11-15 03:35:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-11-15 03:35:41 (120 MB/s) - ‘helper_functions.py’ saved [10246/10246]



### Download the data

In [3]:
# Downloading and preprocessing data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

# Unzip the data
unzip_data("101_food_classes_10_percent.zip")

# Walk through directories
walk_through_dir("101_food_classes_10_percent")

--2023-11-15 03:35:45--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.207, 173.194.203.207, 74.125.199.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  86.1MB/s    in 19s     

2023-11-15 03:36:04 (80.2 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/deviled_eggs'.
There are 0 directories and 

In [4]:
# Split the data into train and test directories
train_dir = "101_food_classes_10_percent/train"
train_dir = "101_food_classes_10_percent/test"

In [5]:
# Create data tensors using data_from_directory
IMG_SIZE = (224,224)
BATCH_SIZE = 32


train_data = tf.keras.preprocessing.image_dataset_from_directory(directory="101_food_classes_10_percent/train",
                                                        label_mode="categorical",
                                                        batch_size=BATCH_SIZE,
                                                        image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory="101_food_classes_10_percent/test",
                                                       label_mode="categorical",
                                                       batch_size=BATCH_SIZE,
                                                       image_size=IMG_SIZE,
                                                       shuffle=False) # We do not shuffle the test data for prediction analysis

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


### Training on 101 food classes with 10% of the data using transfer learning
### First we are going to start with feature extraction model for a few epochs and then fine tune for a few other epochs
#### Things we are going to do beat the original Food101Paper (50.76% accuracy) with 10% of the training data and this modelling setup:
- A modecheckpoint to save progress during training and use this in fine tune method.
- Data Augmentation built right in the model
- A headless EfficientNetB0 model as our base model.
- A Dense layer with 101 hidden neurons with `softmax` as activation.
- `Categorical Crossentropy` as our loss function
- The adam optimizer
- Fitting for 5 epochs and evaluating on 15% of test data


In [ ]:
# Setup checkpoint callback for later use
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint
